# Word embeddings

这部分介绍词嵌入，包括完整代码从一个小型数据集从头训练词嵌入，并利用[Embedding Projector] 可视化这些词嵌入(如下图所示).

<img src="embedding.jpg" alt="Screenshot of the embedding projector" width="400"/>

## 将文本表示为数字

机器学习模型将向量或数字数组作为输入，当处理文本时，第一件事情就是在将文本喂给模型前，给出一种将文本转换成数字或向量化文本的策略，这一节我们将看看三种不同的策略。

### One-hot编码

第一个想法，我们可以对词典中每个词进行单热编码。考虑句子 "The cat sat on the mat". 这个句子的词典或唯一词为(cat, mat, on, sat, the). 为了表示这些词，我们创建一个长度等于词典大小的0向量，然后将词的对应位置置1.这种方法如下图所示.

<img src="one-hot.png" alt="Diagram of one-hot encodings" width="400" />

为了创建一个包含句子编码的向量，我们将每个词的one-hot向量拼接起来。

要点：这种方法低效。单热编码向量是稀疏的，大多数位置为0.

### 将每个单词编码为唯一数字

第二种方法是将每个词用唯一整数标识。继续前面的例子，我们用1标识cat，2标识mat，依此类推。我们可以将句子"The cat sat on the mat"编码为一个密集向量 [5, 1, 4, 3, 5, 2]. 这种方法是高效的，句子的表示是一个密集向量，所有元素都是满的。.

但是，这种方法有两个问题:

* 整数编码是随意，它没有捕捉词之间的任何关系.

* 整数编码的解释对于模型来说是个挑战。例如，线性分类器对于每个特征学习一个权重，因为单词或编码间没有相似性，特征权重的组合就变得没有意义。

### 词嵌入Word embeddings

词嵌入Word embeddings是一种高效和密集的表示，相似的词有相似的编码。重要的是，我们不需要手动指定这种编码。一个嵌入向量是一个浮点数密集向量，向量长度可以作为参数指定。不是手动指定嵌入向量的值，它们是可以学习的参数（或由模型在训练中学习的权重，如同模型学习的其他如密集层的权重一样）。通常，词嵌入维度从8维（小型数据集）到1024维（大型数据集）。较高维度的词嵌入可以捕捉词之间更细粒度的关系，但是也需要更多的训练数据。

<img src="embedding2.png" alt="Diagram of an embedding" width="400"/>

上图是一个词嵌入图。每个词表示为一个4维浮点向量。也可以从另外一个角度理解词嵌入，将嵌入看作是一个查找表。当词嵌入权重学习完后，我们可以从表中查找每个词的编码或嵌入向量。

## 设置

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

## 使用Embedding层

Keras中使用词嵌入比较容易，让我们来看看嵌入层。

Embedding层可以理解为一个查找表，将整数索引（表示具体的词）映射为密集向量（词嵌入）。词嵌入的维度或宽度是一个可以实验的参数，你可以看看多大维度更适合你的问题，正如同你试验Dense层的神经元个数一样。


In [ ]:
embedding_layer = layers.Embedding(1000, 5)

当你创建Embedding层时，权重或嵌入随机初始化（和其他层一样）。在训练中，它们通过反向传播逐渐调整。一旦训练完成，学习到的词嵌入将大致编码词间的相似性。

如果你传一个整数给嵌入层，整数将被嵌入表中的向量替代。

In [ ]:
result = embedding_layer(tf.constant([1,2,3]))
result.numpy()

对于文本或序列问题，嵌入层输入是一个形状为`(samples, sequence_length)`的2维整数向量，每个项或每个序列是一个整数序列。嵌入层可以对变长序列嵌入。形状为`(32, 10)`的批表示批大小为32，每个序列长度为10.

当嵌入层后的层需要需要固定长度序列时，嵌入层初始化参数input_length可以指定序列的长度。

嵌入层返回的张量比输入多一个轴，传递一个 `(2, 3)` 输入给嵌入层，输出为 `(2, 3, N)`，N为嵌入向量维度。


In [ ]:
result = embedding_layer(tf.constant([[0,1,2],[3,4,5]]))
result.shape

当以序列批作为输入时，嵌入层返回一个3D浮点张量，形状为`(samples, sequence_length, embedding_dimensionality)`.有很多标准方法将变长序列转换成定长表示，你可以在传递给密集层前使用RNN，注意力和池化层。本部分使用池化层，因为它最简单。

## 从头学习词嵌入

本部分将训练一个IMDB评论上的情感分类器。模型将从头学习词嵌入，我们将使用一个预处理后的数据集。

In [ ]:
(train_data, test_data), info = tfds.load(
    'imdb_reviews/subwords8k', 
    split = (tfds.Split.TRAIN, tfds.Split.TEST), 
    with_info=True, as_supervised=True)

获得编码器 (`tfds.features.text.SubwordTextEncoder`), 快速看看词典。and have a quick look at the vocabulary. 

词典中的 "\_" 表示空白。注意观察词典如何包含完整词 (以 "\_"结束) 和部分词，部分词可以用来构建更大的词。

In [ ]:
encoder = info.features['text'].encoder
encoder.subwords[:20]

电影评论有不同的长度，我们将使用 `padded_batch`方法来标准化评论的长度。 

In [ ]:
train_data

In [ ]:
train_batches = train_data.shuffle(1000).padded_batch(10, padded_shapes=([None],[]))
test_batches = test_data.shuffle(1000).padded_batch(10, padded_shapes=([None],[]))

注意：在**TensorFlow 2.2**中，参数padded_shapes不再要求。缺省行为是将所有轴填充到批中最大长度。

In [ ]:
train_batches = train_data.shuffle(1000).padded_batch(10)
test_batches = test_data.shuffle(1000).padded_batch(10)

评论文本是整数编码（每个整数表示词典中一个词或部分词）。

由于批根据最长样本填充，注意尾部的0.

In [ ]:
train_batch, train_labels = next(iter(train_batches))
train_batch.numpy()

### 创建一个简单模型

我们将使用Keras序列API定义模型。在这种情况下，它是一个连续词袋风格的模型。

* 嵌入层以整数编码文本为输入，通过查找获得每个词的嵌入向量。嵌入向量和模型一起训练。输出结果维度为: `(batch, sequence, embedding)`.

* 接下来, 一个GlobalAveragePooling1D层通过对文本序列平均返回一个固定长度的输出向量。这允许模型处理可变长度输入，以一种简单的方式。

* 固定长度的输出向量通过一个16个隐藏单元的全连接Dense层。

* 最后一层是一个有单一输出节点的密集层。使用sigmoid函数，这个层的输出值在0到1间，表示评论正面的概率或置信度。

注意：这个模型没有使用masking，因此，0填充作为输入的一部分，因而填充长度可能会影响输出。

In [ ]:
embedding_dim=16

model = keras.Sequential([
  layers.Embedding(encoder.vocab_size, embedding_dim),
  layers.GlobalAveragePooling1D(),  # 对不定长序列中嵌入求平均
  layers.Dense(16, activation='relu'),
  layers.Dense(1)
])

model.summary()

### 编译和训练模型

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(
    train_batches,
    epochs=10,
    validation_data=test_batches, validation_steps=20)

使用这种方法，我们的模型获得88%的验证精度。注意，这个模型有过拟合，因为训练精度高得多。

In [ ]:
import matplotlib.pyplot as plt

history_dict = history.history

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss=history_dict['loss']
val_loss=history_dict['val_loss']

epochs = range(1, len(acc) + 1)

plt.figure(figsize=(12,9))
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.figure(figsize=(12,9))
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim((0.5,1))
plt.show()

## 检索学到的词嵌入

接下来，我们检索训练中学到的词嵌入，它的形状为 `(vocab_size, embedding-dimension)`.

In [ ]:
e = model.layers[0]  # 顺序模型的第一层，即嵌入层
weights = e.get_weights()[0]  # 权重列表取第一个
print(weights.shape) # shape: (vocab_size, embedding_dim)

我们将将权重写到磁盘。使用[Embedding Projector], 我们将上传两个以tab分隔的文件：一个包括嵌入的向量文件，一个包含词的元数据文件。

In [ ]:
import io

encoder = info.features['text'].encoder

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for num, word in enumerate(encoder.subwords):
  vec = weights[num+1] # skip 0, it's padding.
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
out_v.close()
out_m.close()

## 可视化词嵌入

为了可视化嵌入，我们将上传文件到Embedding Projector。

打开[Embedding Projector](http://projector.tensorflow.org/) (这也可以在本地TensorBoard 中运行.).

* 点击"Load data".

* 上传前面创建的两个文件: `vecs.tsv` 和 `meta.tsv`.

你训练的嵌入将显示。你可以搜索单词最近的邻居，例如，搜索"beautiful"，你可能发发现像"wonderful"这样的邻居. 

注意：你的结果可能有些不同，这依赖训练前嵌入层权重的随机初始化。

注意：在实验中，你可以使用简单模型以获得更可解释的嵌入。试试删除 `Dense(16)` 层, 重新训练模型，然后可视化嵌入。

<img src="embedding.jpg" alt="Screenshot of the embedding projector" width="400"/>


## 下一步


本节展示了如何在小型数据集上从头训练和可视化词嵌入。下一步将在文本分类中使用词嵌入。